In [1]:
!pip uninstall tiktoken
!pip uninstall langchain
!pip uninstall pypdf
!pip uninstall sentence-transformers

!pip install tiktoken
!pip install langchain==0.1.7
!pip install pypdf
!pip install sentence-transformers

Found existing installation: tiktoken 0.7.0
Uninstalling tiktoken-0.7.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/tiktoken-0.7.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tiktoken/*
    /usr/local/lib/python3.10/dist-packages/tiktoken_ext/openai_public.py
Proceed (Y/n)? Y
  Successfully uninstalled tiktoken-0.7.0
Found existing installation: pypdf 4.3.1
Uninstalling pypdf-4.3.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pypdf-4.3.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pypdf/*
Proceed (Y/n)? Y
  Successfully uninstalled pypdf-4.3.1
Found existing installation: sentence-transformers 3.0.1
Uninstalling sentence-transformers-3.0.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/sentence_transformers-3.0.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/sentence_transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled sentence-transformers-3.0.1
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_

In [2]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [3]:
loader = PyPDFLoader("/content/DJI_Osmo_Pocket_3_User_Manual_v1.0_en.pdf")
pages = loader.load_and_split()

In [4]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function= tiktoken_len,
)


In [5]:
docs = text_splitter.split_documents(pages)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.1 MB/s eta 0:00:00


In [9]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, hf)

In [16]:
query = '4k recording'
docs = db.similarity_search(query)

print(docs[0])
print('Total count' + str(len(docs))) # 40페이지의 pdf에서 4k recording과 관련된 문서가 4개 있다.
print('First chunk'+ docs[0].page_content)

page_content='Slow Motion 4K: 3840×2160@100/120fps   \n2.7K: 2688×1512@120fps\n1080p: 1920×1080@120/240fps\nHyperlapse 4K/2.7K/1080p@25/30fps: Auto/×2/×5/×10/×15/×30\nTimelapse 4K/2.7K/1080p@25/30fps\nIntervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s\nDuration: 5/10/20/30 mins, 1/2/3/5/∞ hours\nMotionlapse 4K/2.7K/1080p@25/30fps\nIntervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s\nDuration: 5/10/20/30 mins, 1/2/3/5 hours\nSupports setting four positions\nLow-Light Video 4K (16:9): 3840×2160@24/25/30fps\n1080p: 1920×1080@24/25/30fps\nMax Video Bitrate 130 Mbps\nSupported File System exFAT\nPhoto Format JPEG/RAW\nVideo Format MP4 (H.264/HEVC)\nBuilt-in Storage \nCapacityThe camera does not have built-in storage, but storage capacity can \nbe expanded by inserting a microSD card.\nAudio Output 48 kHz 16-bit; AAC\nBattery\nType LiPo\nCapacity 1300 mAh\nEnergy 10.01 Wh' metadata={'source': '/content/DJI_Osmo_Pocket_3_User_Manual_v1.0_en.pdf', 'page': 37}
Total count4
First chunkSlow Moti

In [15]:
for doc in docs:
  print(doc) # 메타데이터에 해당 문서의 pdf 페이지를 알 수 있다.

page_content='Slow Motion 4K: 3840×2160@100/120fps   \n2.7K: 2688×1512@120fps\n1080p: 1920×1080@120/240fps\nHyperlapse 4K/2.7K/1080p@25/30fps: Auto/×2/×5/×10/×15/×30\nTimelapse 4K/2.7K/1080p@25/30fps\nIntervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s\nDuration: 5/10/20/30 mins, 1/2/3/5/∞ hours\nMotionlapse 4K/2.7K/1080p@25/30fps\nIntervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s\nDuration: 5/10/20/30 mins, 1/2/3/5 hours\nSupports setting four positions\nLow-Light Video 4K (16:9): 3840×2160@24/25/30fps\n1080p: 1920×1080@24/25/30fps\nMax Video Bitrate 130 Mbps\nSupported File System exFAT\nPhoto Format JPEG/RAW\nVideo Format MP4 (H.264/HEVC)\nBuilt-in Storage \nCapacityThe camera does not have built-in storage, but storage capacity can \nbe expanded by inserting a microSD card.\nAudio Output 48 kHz 16-bit; AAC\nBattery\nType LiPo\nCapacity 1300 mAh\nEnergy 10.01 Wh' metadata={'source': '/content/DJI_Osmo_Pocket_3_User_Manual_v1.0_en.pdf', 'page': 37}
page_content='the interval and du

### 설명!!
- 자신이 알고싶은 pdf파일을 벡터 데이터베이스에 올려놓고 관련 질문을 서치한 문서를
- 다시 llm 모델에 입력하면, 이상한곳에서 데이터를 끌어오는것이 아닌 주어진 데이터베이스와 데이터베이스에서 출력한 관련 쿼리 문서를 바탕으로
- 대답을 받을 수 있다. 즉 , 정확한 팩트를 요약 및 설명해서 출력하는데 사용가능 !!!

In [17]:

query = 'Who is DJI?'
# similarity_search_with_relevance_scores 는 데이터베이스에서 쿼리와 가장 유사한 문서를 검색하는 함수
docs = db.similarity_search_with_relevance_scores(query, k=3) # k=3 은 검색된 문서 중 상위 3개만 반환
docs[0][0] # docs[0]는 관련성이 가장 높은 첫 번째 문서와 그 점수를 나타냅니다
# docs[0][0]은 해당 문서 자체를 반환

/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:311: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='If you have any questions about this document, please \ncontact DJI by sending a message to DocSupport@dji.com .\nDJI and OSMO are trademarks of DJI. \nCopyright © 2023 DJI OSMO All Rights Reserved.WE ARE HERE FOR YOU\nContact \nDJI SUPPORT\nhttps://www.dji.com/osmo-pocket-3/downloadsThis content is subject to change.', metadata={'source': '/content/DJI_Osmo_Pocket_3_User_Manual_v1.0_en.pdf', 'page': 39}), -229.15829406334055), (Document(page_content='DJI Osmo Pocket 3 User Manual© 2023 DJI All Rights Reserved. \u200327DJI Mic 2 Transmitter Status LEDs\n1 2\nDJI Mic 2 Transmitter Connection\nFollow the steps below to connect to DJI Mic 2 (hereinafter referred to as “transmitter”).\n1. Swipe down from the top of the screen and enter the control menu. Tap the settings icon \nand select Wireless Mic > TX1/TX2, and the camera is rea

Document(page_content='If you have any questions about this document, please \ncontact DJI by sending a message to DocSupport@dji.com .\nDJI and OSMO are trademarks of DJI. \nCopyright © 2023 DJI OSMO All Rights Reserved.WE ARE HERE FOR YOU\nContact \nDJI SUPPORT\nhttps://www.dji.com/osmo-pocket-3/downloadsThis content is subject to change.', metadata={'source': '/content/DJI_Osmo_Pocket_3_User_Manual_v1.0_en.pdf', 'page': 39})

In [18]:
db.save_local('/content/faiss_db') # 데이터 베이스 저장 => 안하면 메모리에 있는 데이터베이스는 없어져 버림

In [21]:
# 저장된 데이터 베이스 를 메모리로 불러오기 , 저장때와 동일한 임베딩 모델을 지정해야 함
new_db = FAISS.load_local('/content/faiss_db', hf)

query = 'Who is DJI?'

docs = new_db.similarity_search(query)
print(docs[0])

page_content='If you have any questions about this document, please \ncontact DJI by sending a message to DocSupport@dji.com .\nDJI and OSMO are trademarks of DJI. \nCopyright © 2023 DJI OSMO All Rights Reserved.WE ARE HERE FOR YOU\nContact \nDJI SUPPORT\nhttps://www.dji.com/osmo-pocket-3/downloadsThis content is subject to change.' metadata={'source': '/content/DJI_Osmo_Pocket_3_User_Manual_v1.0_en.pdf', 'page': 39}
